In [9]:
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv, to_numeric
from sklearn import cluster
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import math

Q1: Parametric Classification: Using each of the 64 input features separately as the single input dimension, use parametric classification, assuming that the input is distributed according to a Gaussian. Report the training and test errors for the case of each of the 64 features. Which feature(s) give the best test performance? 

In [10]:

XT = read_csv('/media/anuja/study/Anuja/MS/CS_697A_ML/hw2/optdigits_69_N200.tra', sep=",")
XT

,0,0.1,5,14,4,0.2,0.3,0.4,0.5,0.6,...,0.27,0.28,0.29,4.6,12.1,14.4,7,0.30,0.31,6.1
0,0,0,0,10,12,0,0,0,0,0,...,0,0,0,1,11,14,12,1,0,6
1,0,0,2,14,10,0,0,0,0,1,...,0,0,0,1,12,14,12,4,0,6
2,0,0,2,13,5,0,0,0,0,0,...,0,0,0,4,15,12,9,1,0,6
3,0,0,2,13,6,0,0,0,0,0,...,0,0,0,2,12,14,12,4,0,6
4,0,0,1,12,11,0,0,0,0,0,...,2,0,0,2,14,15,12,3,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0,0,0,8,13,10,5,0,0,0,...,0,0,0,0,14,15,2,0,0,9
195,0,0,0,3,10,13,12,4,0,0,...,0,0,0,0,1,15,1,0,0,9
196,0,0,0,4,10,13,12,0,0,0,...,0,0,0,0,2,11,0,0,0,9
197,0,0,6,14,16,9,0,0,0,5,...,0,0,0,8,14,16,13,1,0,9


In [11]:
# # we have 64 attributes + 1 classification column
# # all features in x train
# X_train =  XT.iloc[:,1:64]
# print(X_train)
# # outcome or label in Y train
# Y_train = XT.iloc[:,64]
# print(Y_train)


In [12]:
# use XT as numpy array

myArray = np.asarray(XT)
print(myArray)
[totrows, totcols] = myArray.shape

# now convert split the arrays into two parts as we have two classes here
class1 = myArray[myArray[:,64] == 6, :]
print(class1)

class2 = myArray[myArray[:,64] == 9, :]
print(class2)

[[0 0 0 ... 1 0 6]
 [0 0 2 ... 4 0 6]
 [0 0 2 ... 1 0 6]
 ...
 [0 0 0 ... 0 0 9]
 [0 0 6 ... 1 0 9]
 [0 0 2 ... 4 0 9]]
[[0 0 0 ... 1 0 6]
 [0 0 2 ... 4 0 6]
 [0 0 2 ... 1 0 6]
 ...
 [0 0 0 ... 7 0 6]
 [0 0 2 ... 4 0 6]
 [0 0 1 ... 3 0 6]]
[[0 0 0 ... 0 0 9]
 [0 0 1 ... 0 0 9]
 [0 0 0 ... 0 0 9]
 ...
 [0 0 0 ... 0 0 9]
 [0 0 6 ... 1 0 9]
 [0 0 2 ... 4 0 9]]


In [13]:
# get the occurances or count of both the classes

[rows_of_class1, col1] = class1.shape
print("Number of rows of class 1 : ", rows_of_class1)

[rows_of_class2, col2] = class2.shape
print("Number of rows of class 2 : ",rows_of_class2)



Number of rows of class 1 :  99
Number of rows of class 2 :  100


In [14]:
# now will fine out the class priors

p1 = rows_of_class1/totrows
print("Prior probability of class 1 : ", p1)

p2 = rows_of_class2/totrows
print("Prior probability of class 2 : ", p2)


Prior probability of class 1 :  0.49748743718592964
Prior probability of class 2 :  0.5025125628140703


In [15]:
# now find the mean and std deviation for each feature of the classes (6 and 9). We have 64 features here and 2 classes.

df = pd.DataFrame(columns = ["Mean_class1", "Std_Deviation_class1", "Mean_class2", "Std_Deviation_class2"])

for icol in range(col1-1):
    mean_class1 = np.mean(class1[:,icol])
    std_dv_class1 = np.std(class1[:,icol])

    mean_class2 = np.mean(class2[:,icol])
    std_dv_class2 = np.std(class2[:,icol])
    
    df = df.append({'Mean_class1': mean_class1, 'Std_Deviation_class1': std_dv_class1, 'Mean_class2': mean_class2, 'Std_Deviation_class2': std_dv_class2}, ignore_index=True)


print(df)

    


    Mean_class1  Std_Deviation_class1  Mean_class2  Std_Deviation_class2
0      0.000000              0.000000         0.00              0.000000
1      0.000000              0.000000         0.10              0.331662
2      1.585859              2.912911         4.32              3.627341
3     10.353535              3.412139        10.99              4.875438
4      8.212121              5.288204        11.44              3.066985
..          ...                   ...          ...                   ...
59     9.989899              2.996614        11.35              5.324237
60    14.626263              1.374999        12.42              4.235989
61    13.616162              2.627389         7.45              5.936961
62     6.525253              4.346933         1.61              3.252368
63     0.292929              1.075461         0.05              0.497494

[64 rows x 4 columns]


In [17]:
# calculating p(x | Ci)
# # %debug
# p_class1 = np.empty((rows_of_class1, col1-1),float)
# for icol in range(col1-1):
#     mean = df.Mean_class1[icol]
#     std = df.Std_Deviation_class1[icol]
#     p_feature = []
#     for irow in range (rows_of_class1):
#         exponent = (-(class1[irow,icol] - mean)** 2)/(2*(std**2))
#         p_feature.append((1/(math.sqrt(2*math.pi)*std))*(math.exp(exponent)))
#     p_class1 = np.append(p_class1, np.array([p_feature]).transpose(), axis=1)


p_class1 = np.empty((rows_of_class1, col1-1),float)
for icol in range(col1-1):
    mean = df.Mean_class1[icol]
    std = df.Std_Deviation_class1[icol]
    descriminant_of_feature = []
    for irow in range (rows_of_class1):
        descriminant_of_feature.append(-((1/2)* math.log(2*math.pi)) - math.log(std) - ((class1[irow,icol] - mean)** 2)/(2*(std**2)) + math.log(p1))
    p_class1 = np.append(p_class1, np.array([descriminant_of_feature]).transpose(), axis=1)
    

ValueError: math domain error

Q2: Use all the 64 features, assume that inputs are 64 dimensional Gaussians, and assume that for each class the covariance matrix is different. Report the training and test confusion matrices and errors. Hint: eliminate features that have variance zero. 

Q3: Repeat Q2, assuming that all the class covariance matrices are the same. 

Q4: Use the first 10 features in Q1 that gave the best test performance and repeat Q2. Compare the test error performance you got to Q2. 